In [1]:
import time
import json
from os import listdir
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
## ABRIR O ULTIMO DB

lotofacil_db_dir = './db/lotofacil/'
dbs = listdir(lotofacil_db_dir)
filename = lotofacil_db_dir+dbs[-1]
with open(filename, 'r') as openfile: 
    db = json.load(openfile)

In [3]:
## Selenium Configuration and actuation ##

opts = Options()
# opts.add_argument("user-agent=[Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) \
#                     Chrome/79.0 Safari/537.36]")

opts.add_argument('headless')

browser = webdriver.Chrome(options=opts)

browser.get("http://loterias.caixa.gov.br/wps/portal/loterias/landing/lotofacil/")
ultimo_concurso = browser.find_element_by_xpath('//*[@id="resultados"]/div[1]/div/h2/span').text
ultimo_concurso = (ultimo_concurso.split())[1]

# O formato da chave é "Concurso NNN (dd/mm/aaaa)"
def find_concurso(dica, db):
    found_key = None
    for key in db.keys():
        if str(dica) in key:
            found_key = key
    if found_key == None:
        found_key = "Concurso não encontrado."
    return found_key
    
def find_buraco(db, ultimo_concurso):
    concurso = 1
    faltantes = []
    while concurso <= int(ultimo_concurso):
        if find_concurso('Concurso '+str(concurso), db) in 'Concurso não encontrado.':
            faltantes.append(concurso)
        concurso+=1
    return faltantes


faltantes = find_buraco(db, ultimo_concurso)
print('\nNão foram encontrados os concursos:', faltantes, '\n\nUm total de:', len(faltantes))


resultados = {}

for faltante in faltantes:
    busca = browser.find_element_by_xpath('//*[@id="buscaConcurso"]')
    busca.send_keys(Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE)
    busca.send_keys(faltante)
    while str(faltante) not in (browser.find_element_by_xpath('//*[@id="resultados"]/div[1]/div/h2/span')).text:
        busca.send_keys(Keys.ENTER)
        time.sleep(1)
    concurso = (browser.find_element_by_xpath('//*[@id="resultados"]/div[1]/div/h2/span')).text
    numeros = []
    numeros.append((browser.find_element_by_xpath('//*[@id="resultados"]/div[2]/div/div/div[1]/table')).text)
    numeros = numeros[0].split()
    numeros = [int(i) for i in numeros]
    
    resultados[concurso] = {
        'Numeros': numeros
    }
    print(concurso, numeros)
browser.quit()
print('Done.')

## ## ##


Não foram encontrados os concursos: [1965] 

Um total de: 1
Concurso 1965 (11/05/2020) [1, 2, 4, 5, 6, 10, 12, 16, 19, 20, 21, 22, 23, 24, 25]
Done.


In [4]:
## SALVAR NOVO DB
db.update(resultados)
dictionary = db
lotofacil_db_dir = './db/lotofacil/'
filename = './db/lotofacil/lotofacil_'+str(int(time.time()))+'.json'
with open(filename, "w") as outfile: 
    json.dump(dictionary, outfile)